In [124]:
import pandas as pd 
import numpy as np 
import math 
from sklearn.linear_model import LinearRegression
import sklearn.metrics
from sklearn.preprocessing import StandardScaler

# MEN'S ANALYSIS

## Reading in the data

In [118]:
# Men's regular season detailed stats
MRegStats = pd.read_csv("Data/MRegularSeasonDetailedResults.csv")

# Men's regular season compact stats
MCompStats = pd.read_csv("Data/MRegularSeasonCompactResults.csv")

# Men's NCAA tournament detailed stats
MTournStats = pd.read_csv("Data/MNCAATourneyDetailedResults.csv")

# Men's NCAA tournament compact stats
MCompTournStats = pd.read_csv("Data/MNCAATourneyCompactResults.csv")

# Men's Conference tournament stats
MConfTournStats = pd.read_csv("Data/MConferenceTourneyGames.csv")

# Men's Team names
MTeams = pd.read_csv("Data/MTeams.csv")
MTeamSpellings = pd.read_csv("Data/MTeamSpellings.csv", encoding='unicode_escape')

# Men's Massey Ordinals
MOridinals = pd.read_csv("Data/MMasseyOrdinals.csv")

# Men's Tournament Seeds
MSeeds = pd.read_csv("Data/MNCAATourneySeeds.csv")

# Men's Coaches
MCoaches = pd.read_csv("Data/MTeamCoaches.csv")

# Men's Conferences
MConferences = pd.read_csv("Data/MTeamConferences.csv")

# Cities
Cities = pd.read_csv("Data/Cities.csv")
MGameCities = pd.read_csv("Data/MGameCities.csv")

In [119]:
# Basic Percentage Stats
# FG Perc
MRegStats["WFGPerc"] = MRegStats["WFGM"] / MRegStats["WFGA"]
MRegStats["LFGPerc"] = MRegStats["LFGM"] / MRegStats["LFGA"]

# 3PT Perc
MRegStats["WFG3Perc"] = MRegStats["WFGM3"] / MRegStats["WFGA3"]
MRegStats["LFG3Perc"] = MRegStats["LFGM3"] / MRegStats["LFGA3"]

# 3PT Rate
MRegStats["W3Rate"] = MRegStats["WFGA3"] / MRegStats["WFGA"]
MRegStats["L3Rate"] = MRegStats["LFGA3"] / MRegStats["LFGA"]

# FT Perc
#MRegStats["WFTPerc"] = MRegStats["WFTM"] / MRegStats["WFTA"]
#MRegStats["LFTPerc"] = MRegStats["LFTM"] / MRegStats["LFTA"]

# 2PT Perc
MRegStats["WFG2Perc"] = (MRegStats["WFGM"] - MRegStats["WFGM3"]) / (MRegStats["WFGA"] - MRegStats["WFGA3"])
MRegStats["LFG2Perc"] = (MRegStats["LFGM"] - MRegStats["LFGM3"]) / (MRegStats["LFGA"] - MRegStats["LFGA3"])

# 2PT Rate
MRegStats["W2Rate"] = (MRegStats["WFGA"] - MRegStats["WFGA3"]) / MRegStats["WFGA"]
MRegStats["L2Rate"] = (MRegStats["LFGA"] - MRegStats["LFGA3"]) / MRegStats["LFGA"]


In [120]:
# Tempo
# This is how the NET rankings calculate tempo so we will emulate it
MRegStats["WTempo"] = MRegStats["WFGA"] - MRegStats["WOR"] + MRegStats["WTO"] + 0.475 * MRegStats["WFTA"]
MRegStats["LTempo"] = MRegStats["LFGA"] - MRegStats["LOR"] + MRegStats["LTO"] + 0.475 * MRegStats["LFTA"]

# Four Factors
# eFG%
MRegStats["WeFG"] = (MRegStats["WFGM"] + 0.5 * MRegStats["WFGM3"]) / MRegStats["WFGA"]
MRegStats["LeFG"] = (MRegStats["LFGM"] + 0.5 * MRegStats["LFGM3"]) / MRegStats["LFGA"]

# OR%
MRegStats["WORPerc"] = MRegStats["WOR"] / (MRegStats["WOR"] + MRegStats["LDR"])
MRegStats["LORPerc"] = MRegStats["LOR"] / (MRegStats["LOR"] + MRegStats["WDR"])

# TO%
MRegStats["WTOPerc"] = (MRegStats["WTO"] / MRegStats["WTempo"]) * 100
MRegStats["LTOPerc"] = (MRegStats["LTO"] / MRegStats["LTempo"]) * 100

# FTR
MRegStats["WFTR"] = MRegStats["WFTA"] / MRegStats["WFGA"]
MRegStats["LFTR"] = MRegStats["LFTA"] / MRegStats["LFGA"]

In [121]:
# Net Rating Stats

# "uORTG" is unadjusted Offensive Rating. This will be used to adjust later.
MRegStats["WuORTG"] = (MRegStats["WScore"] / MRegStats["WTempo"]) * 100
MRegStats["LuORTG"] = (MRegStats["LScore"] / MRegStats["LTempo"]) * 100

# The same goes for defensive rating. "uDRTG" is the unadjusted defensive rating
MRegStats["WuDRTG"] = (MRegStats["LScore"] / MRegStats["LTempo"]) * 100
MRegStats["LuDRTG"] = (MRegStats["WScore"] / MRegStats["WTempo"]) * 100

# And now Net Rating is just Offensive Rating - Defensive Rating
MRegStats["WuNetRtg"] = MRegStats["WuORTG"] - MRegStats["WuDRTG"]
MRegStats["LuNetRtg"] = MRegStats["LuORTG"] - MRegStats["LuDRTG"]


Adjusting the data to be longer instead of wider. Twice as long but groupable by team now.

In [122]:
# Making two dataframes so that they can be added to each other when they change
MWTeamStats = MRegStats.copy()
MLTeamStats = MRegStats.copy()
# Changing location of teams between wins and losses
MLTeamStats["WLoc"] = MLTeamStats["WLoc"].map(lambda x: "H" if x == "A" else "A" if x == "H" else x)
MLTeamStats["LLoc"] = MLTeamStats["WLoc"]
MLTeamStats.drop("WLoc",axis = 1, inplace = True)


# Rename columns for MWTeamStats (Team perspective)
MWTeamStats.rename(columns={col: col.replace("W", "Team_", 1) for col in MWTeamStats.columns if col.startswith("W")}, inplace=True)
MWTeamStats.rename(columns={col: col.replace("L", "Opp_", 1) for col in MWTeamStats.columns if col.startswith("L")}, inplace=True)

# Rename columns for MLTeamStats (Opponent perspective)
MLTeamStats.rename(columns={col: col.replace("W", "Opp_", 1) for col in MLTeamStats.columns if col.startswith("W")}, inplace=True)
MLTeamStats.rename(columns={col: col.replace("L", "Team_", 1) for col in MLTeamStats.columns if col.startswith("L")}, inplace=True)

# Final Team Stats dataframe
MTeamStats = pd.concat([MWTeamStats, MLTeamStats], ignore_index=True)

Additional Stats Added - Conference (Name and Record), Coach, Wins, City (later)

In [ ]:
# Merge conference names for conference bias later in the analysis
MTeamStats = MTeamStats.merge(MConferences,left_on = ["Season","Team_TeamID"],right_on = ["Season","TeamID"]).rename(columns = {"ConfAbbrev": "Team_Conf"})
MTeamStats = MTeamStats.merge(MConferences, left_on = ["Season","Opp_TeamID"], right_on = ["Season","TeamID"]).rename(columns = {"ConfAbbrev": "Opp_Conf"})
MTeamStats["ConfGame"] = (MTeamStats["Team_Conf"] == MTeamStats["Opp_Conf"]).astype(int)

MTeamStats.head()

,Season,DayNum,Team_TeamID,Team_Score,Opp_TeamID,Opp_Score,Team_Loc,NumOT,Team_FGM,Team_FGA,...,Opp_uORTG,Team_uDRTG,Opp_uDRTG,Team_uNetRtg,Opp_uNetRtg,TeamID_x,Team_Conf,TeamID_y,Opp_Conf,ConfGame
0,2003,10,1104,68,1328,62,N,0,27,58,...,86.773968,86.773968,90.006618,3.232650,-3.232650,1104,sec,1328,big_twelve,0
1,2003,98,1400,67,1328,61,H,0,25,64,...,95.126706,95.126706,102.290076,7.163371,-7.163371,1400,big_twelve,1328,big_twelve,1
2,2003,124,1400,76,1328,71,A,0,27,50,...,109.356950,109.356950,115.370019,6.013069,-6.013069,1400,big_twelve,1328,big_twelve,1
3,2003,111,1242,70,1328,77,A,0,21,61,...,107.354479,107.354479,96.319229,-11.035249,11.035249,1242,big_twelve,1328,big_twelve,1
4,2003,120,1304,51,1328,76,A,0,21,53,...,108.455227,108.455227,72.237960,-36.217266,36.217266,1304,big_twelve,1328,big_twelve,1


In [ ]:
# Add a result to see if a team won or not; this will be used for win percentage later
MTeamStats["Result"] = (MTeamStats["Team_Score"] > MTeamStats["Opp_Score"]).astype(int)

Instead of looking at just net rating, we want to adjust based on the offensive and defensive ratings as well. We normalize the offense and defensive ratings in order to get an adjustment for later.

In [142]:
# Select only numerical columns in the MTeamStats dataframe
TeamStatsNumericCols = MTeamStats.select_dtypes(include = ['number']).columns.difference(["Season","Team_TeamID","Opp_TeamID"])

# Use the numerical columns to then get the mean of all of the columns
MTeamGroupedStats = MTeamStats.groupby(["Season","Team_TeamID","Team_Conf"])[TeamStatsNumericCols].mean().reset_index()

# Group by conference to get conference adjustments
MConfNetStats = MTeamStats.groupby(["Season","Team_Conf"])[["Team_uORTG","Team_uDRTG","Team_uNetRtg"]].mean().reset_index()
MConfNetStats["Team_uORTGweight"] = MConfNetStats["Team_uORTG"] / MConfNetStats["Team_uORTG"].mean()
MConfNetStats["Team_uDRTGweight"] = MConfNetStats["Team_uDRTG"] / MConfNetStats["Team_uDRTG"].mean()

In [ ]:
# Merging the conference weights back into the original dataframe
MTeamGroupedStats = MTeamGroupedStats.merge(MConfNetStats[["Season","Team_Conf","Team_uORTGweight","Team_uDRTGweight"]],on = ["Season","Team_Conf"])

In [150]:
# Creating the conference-adjusted team weights
MTeamGroupedStats["ConfAdjORTG"] = MTeamGroupedStats["Team_uORTG"] * MTeamGroupedStats["Team_uORTGweight"]
MTeamGroupedStats["ConfAdjDRTG"] = MTeamGroupedStats["Team_uDRTG"] * MTeamGroupedStats["Team_uDRTGweight"]
MTeamGroupedStats["ConfAdjNetRtg"] = MTeamGroupedStats["ConfAdjORTG"] - MTeamGroupedStats["ConfAdjDRTG"]

In [184]:
MTeamGroupedStats.groupby(["Team_TeamID"])["ConfAdjNetRtg"].mean()

Team_TeamID
1101    -6.302463
1102     1.651105
1103     7.152810
1104    13.362988
1105   -17.258504
          ...    
1476   -14.526650
1477   -16.790496
1478   -14.347080
1479   -18.395853
1480   -21.491918
Name: ConfAdjNetRtg, Length: 371, dtype: float64

#### Linear Regression

Now we will look at linear regression for the offensive side and defensive side to see what other adjustments need to be made. Sometimes, irregardless of conference, some teams just outperform or underperform their peers, and should be rewarded or punished respectively.

In [185]:
from sklearn.preprocessing import PolynomialFeatures

lreg = LinearRegression()
MRegDF = MTeamGroupedStats.copy()
MRegDF["Team_TeamID"].astype('category')
lreg_teams = MRegDF[["Team_TeamID","Season"]]
lreg_netrtg = MRegDF["ConfAdjNetRtg"]
lreg.fit(lreg_teams,lreg_netrtg)

print(lreg.predict([[140000,2024]]))

[1200.83311642]


c:\Users\mattd\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
